# ConfigState example with training a model using Tensorflow

This notebook presents an example of how the config-state library can be used to design a machine learning experiment that consists in training an image classification model. We show how the different components, the dataset, the model and the optimizer can be configured and modified through a config file without requiring to write code. We also show how the experiment can be saved at regular intervals and be resumed in case of interruption.

### Requirements

The packages `tensorflow` and `tensorflow-datasets` are required for this example:
```
pip install tensorflow
pip install tensorflow-datasets
```

### The `MLExperiment` class

The `MLExperiment` class is a `ConfigState` subclass that defines the experiment consisting of training a machine learning model for image classification. It is composed of nested `ConfigState` objects that represent the different components such as `Dataset`, `Model` and `Optimizer`.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # reduce tensorflow's verbosity

from examples.tensorflow.experiment import MLExperiment

### Configuring a `MLExperiment` experiment
The directory `examples/tensorflow/configs` contains examples of configuration files that can be used to configure an experiment. Let's load one:

In [ ]:
import yaml

config = yaml.load(open("tensorflow/configs/mlp.yml", 'r'), Loader=yaml.FullLoader)

An experiment can be instantiated using this configuration:

In [ ]:
experiment = MLExperiment(config)

print(experiment.config_summary())

We can start training the model for a given number of epochs:

In [ ]:
experiment.run(epochs=2)

### Saving and restoring an experiment

The current experiment's state can been saved into file:

In [ ]:
from config_state import Serializer
import tempfile
from pathlib import Path

# create a temporary directory
temp_dir = tempfile.TemporaryDirectory()

# file that will store the experiment
file_path = Path(temp_dir.name) / 'exp.save'

# save the experiment using the Pickle serializer
Serializer({'class': 'Pickle'}).save(experiment, file_path)

The experiment can be restored and resumed:

In [ ]:
experiment = Serializer({'class': 'Pickle'}).load(file_path)

experiment.run(epochs=2)

temp_dir.cleanup()

### Configuring a new experiment

We can customize the config dictionary to design a new experiment with a different datatet, model or optimizer:

In [ ]:
config['dataset'] = {
    'name': 'cifar10' # https://www.tensorflow.org/datasets/catalog/overview#image_classification
}
config['model'] = {
    'class': 'CNN',
    'structure': [32, 'max', 64, 'max', 64]
}
config['optimizer'] = {
    'class': 'Adam',
    'learning_rate': 0.001
}

experiment = MLExperiment(config)

print(experiment.config_summary())

experiment.run(epochs=20)

### ConfigState objects composability

`ConfigState` is convenient for compositing objects. For instance we can nest a `Model` into another `Ensembler` model:

In [ ]:
cnn_model = {
    'class': 'CNN',
    'structure': [32, 'max', 64, 'max', 64]
}

config['model'] = {
    'class': 'Ensembler',
    'model': cnn_model,
    'ensemble_size': 4
}

config['dataset'] = {
    'name': 'cifar10',
    'batch_size': 128 # We augment the batch_size so that each ensembled models train on batches of 32 elements
}

experiment = MLExperiment(config)

print(experiment.config_summary())

print(experiment.model.keras_model.summary())


In [ ]:
experiment.run(epochs=20)

Since `Ensembler` is itself a `Model`, we can compose it into another `Ensemble` such that we can define models that are ensemble of ensemble:

In [ ]:
cnn_model = {
    'class': 'CNN',
    'structure': [32, 'max', 64, 'max', 64]
}

ensemble = {
    'class': 'Ensembler',
    'model': cnn_model,
    'ensemble_size': 4
}

config['model'] = {
    'class': 'Ensembler',
    'model': ensemble,
    'ensemble_size': 4
}

config['dataset'] = {
    'name': 'cifar10',
    'batch_size': 512
}

experiment = MLExperiment(config)

# ensemble_ensemble_exp.model.model.output_units
print(experiment.config_summary())

print(experiment.model.keras_model.summary())


In [ ]:
experiment.run(epochs=1)